In [5]:
import cv2
import time
import datetime
import pandas as pd
import os
import detect_emotion as de
import detect_headpose as dhp
import detect_eye_openness as deo
import matplotlib.pyplot as plt

In [6]:
# Capture image from camera function
def capture_image_from_camera():
    # Initialize the camera capture object
    cap = cv2.VideoCapture(0)

    while True:
        # Capture frame from the camera
        ret, frame = cap.read()

        if ret:
            # Display the captured frame
            cv2.imshow("Camera Feed", frame)

            key = cv2.waitKey(1)

            # Press 'c' to capture the frame
            if key & 0xFF == ord("c"):
                captured_image = frame.copy()

            # Press 'q' to exit the loop
            elif key & 0xFF == ord("q"):
                break
        else:
            print("Failed to capture frame.")
            break

    # Release the camera and close windows
    cap.release()
    cv2.destroyAllWindows()

    return captured_image

In [7]:
#Stores 7 CI and returns the average
def get_EL(CI, CI_List):
    if len(CI_List) == 7:  # If the list is full
        CI_List.pop(0)  # Remove the first CI
        CI_List.append(CI)  # Add the new CI to the last index
    else:  # If the list is not full
        CI_List.append(CI)  # Add the new CI to the last index
    
    # Calculate and return the average of the list
    average = sum(CI_List) / len(CI_List)
    return average

### Sample run to use camera to get the eye gaze weight in the picture.

In [ ]:
captured_image = capture_image_from_camera()
#detect if there is a face
face_detection = deo.detect_faces(captured_image)
if face_detection is not None:
    head_pose = dhp.detect_head_direction(captured_image)
    # print("Head Pose is: ", head_pose)
    prominent_face = deo.get_prominent_face(captured_image, face_detection)
    eye_openness = deo.get_eye_openness(prominent_face)
    eye_gaze_weight = deo.get_eye_gaze_weight(eye_openness, head_pose)

    #get emotion
    emotion = de.detect_emotion(prominent_face)
    print("Emotion is: ", emotion)
    emotion_weight = de.assign_emotion_weight(emotion)
    # print("Eye gaze weight is: ", eye_gaze_weight)
    # print("Emotion weight is: ", emotion_weight)

    CI = (emotion_weight * eye_gaze_weight) / 4.5
    print("CI is: ", CI)
    # Display the captured image using Matplotlib
    plt.imshow(cv2.cvtColor(captured_image, cv2.COLOR_BGR2RGB))
    plt.title("Captured Image")
    plt.axis('off')  # Hide axes
    plt.show()
else:
    CI = 0
    print("No face detected.")
    
    

### Console-based program to record pariticpant and the notification from the system.

In [6]:
# Initialize variables
data_writer = []
CI_List = []
notify = False

# Define the video capture object
cap = cv2.VideoCapture(0)  # 0 for the default camera, change it to the desired camera index if needed

# Define the codec and create a VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'XVID')  # You can change the codec as needed
output = cv2.VideoWriter('output.avi', fourcc, 30.0, (640, 480))  # Change the output file name and resolution

# Set the duration to record (in seconds)
record_duration = 600  # Change this to the desired recording duration

# Get the start time
start_time = time.time()

# Initialize last_second
last_second = int(start_time)

while True:
    ret, frame = cap.read()

    if not ret:
        break

    # Write the frame to the output video file
    output.write(frame)

    # Display the frame (optional)
    cv2.imshow('Recording', frame)

    elapsed_time = time.time() - start_time

    # Check if the recording duration has been reached
    if elapsed_time >= record_duration+1:
        break

    # Check if a second has passed then perform the detection
    current_second = int(elapsed_time)
    if current_second != last_second:
        # Detect if there is a face
        face_detection = deo.detect_faces(frame)
        if face_detection is not None:
           # Get head pose with the original image
            head_pose = dhp.detect_head_direction(frame)
            prominent_face = deo.get_prominent_face(frame, face_detection)
            eye_openness = deo.get_eye_openness(prominent_face)
            eye_gaze_weight = deo.get_eye_gaze_weight(eye_openness, head_pose)

            # Get emotion
            emotion = de.detect_emotion(prominent_face)
            emotion_weight = de.assign_emotion_weight(emotion)

            CI = (emotion_weight * eye_gaze_weight) / 4.5
        else:
            CI = 0
        
        engagement_level = get_EL(CI, CI_List)
        if engagement_level < 0.25:
            notify = True
            data_writer.append([current_second, CI, engagement_level, notify])
            print(datetime.timedelta(seconds=current_second), notify)
        else:
            notify = False
            data_writer.append([current_second, CI, engagement_level, notify])
            print(datetime.timedelta(seconds=current_second), notify) 
            
        last_second = current_second  # Update last_second
    # Check for the 'q' key press to exit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture and writer, and close any open windows
cap.release()
output.release()
cv2.destroyAllWindows()

0:00:00 True
0:00:01 False
0:00:02 False
0:00:03 False
0:00:05 False
0:00:07 False
0:00:08 False
0:00:09 False
0:00:10 False
0:00:11 False
0:00:12 False
0:00:13 False
0:00:14 False
0:00:15 False
0:00:16 False
0:00:18 False
0:00:19 False
0:00:20 False
0:00:21 False
0:00:22 False


In [7]:
data_writer

[[0, 0, 0.0, True],
 [1, 1.0, 0.5, False],
 [2, 0.13333333333333333, 0.37777777777777777, False],
 [3, 0.4, 0.3833333333333333, False],
 [5, 0.19999999999999998, 0.3466666666666666, False],
 [7, 0.30000000000000004, 0.33888888888888885, False],
 [8, 1.0, 0.4333333333333333, False],
 [9, 1.0, 0.5761904761904761, False],
 [10, 0.6666666666666666, 0.5285714285714286, False],
 [11, 1.0, 0.6523809523809524, False],
 [12, 1.0, 0.738095238095238, False],
 [13, 0.19999999999999998, 0.7380952380952381, False],
 [14, 1.0, 0.838095238095238, False],
 [15, 1.0, 0.8380952380952381, False],
 [16, 0.6666666666666666, 0.7904761904761906, False],
 [18, 1.0, 0.8380952380952381, False],
 [19, 1.0, 0.8380952380952381, False],
 [20, 1.0, 0.8380952380952381, False],
 [21, 0.30000000000000004, 0.8523809523809522, False],
 [22, 1.0, 0.8523809523809522, False]]

In [ ]:
# Save results to a CSV file
df = pd.DataFrame(data_writer, columns=["Seconds", "CI", "Engagement", "Notify"])

folder_name = "../NotificationTestingResults"
# Specify the full path to the CSV file within the folder
csv_filename = os.path.join(folder_name, "TestResult10.csv")

# Save the DataFrame to a CSV file
df.to_csv(csv_filename, index=False)

print("Test results saved to ", csv_filename)